In [2]:
import os, getpass


# Get environment variables if not set
def _set_env(var: str):
        if not os.environ.get(var):
            os.environ[var] = getpass.getpass(f'Enter your {var} API key: ')

# Check necessary environment variables
_set_env('TAVILY_API_KEY')
_set_env('LANGCHAIN_API_KEY')

os.environ['TOKENIZERS_PARALLELISM'] = 'true'
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_PROJECT'] = 'langchain-rag-ollama'

### Search

# from langchain_community.tools.tavily_search import TavilySearchRun

In [3]:
## Load the model

from langchain_ollama import ChatOllama

local_llm = 'llama3.2'
llm = ChatOllama(model=local_llm, temperature=0.0)
llm_json_mode = ChatOllama(model=local_llm, temperature=0.0, format='json')

In [6]:
## Logger testing

import logging

logging.basicConfig(
    level=logging.INFO,     
    format="%(levelname)s | %(asctime)s | %(message)s",
    datefmt="%Y-%m-%dT%H:%M:%SZ"
)

logger = logging.getLogger(__name__)

logger.info("Hello, world!")

INFO | 2025-01-07T17:38:45Z | Hello, world!


### Vector store and embedding model
*Using basic query to test the vectorstore and embedding model.*

---------------------------

In [11]:
from src.embedding.vector_store import VectorStore
from src.embedding.embedding_models import EmbeddingModels

embedding_model = EmbeddingModels().get_bge_embedding(model_name="BAAI/bge-base-en") # use default model
chroma_kb = VectorStore.get_chroma_vectorstore(
    vectorstore_path='./data/vector_database/calm_kb', 
    embedding=embedding_model
)

# query_text = 'what is ADRD' # failed, no results
query_text = 'how to take care of ADRD parents' 
similarity_search_res = VectorStore.retrieve_docs(query=query_text, vectorstore=chroma_kb, k=5)

for doc in similarity_search_res:
    print(doc.page_content)
    print(doc.metadata)

2025-01-07 @ 17:43:32 | INFO     | src.embedding.embedding_models:get_bge_embedding:28 - Using device: mps
2025-01-07 @ 17:43:32 | INFO     | src.embedding.embedding_models:get_bge_embedding:29 - Loading BGE embedding model: BAAI/bge-base-en


INFO | 2025-01-07T17:43:32Z | Load pretrained SentenceTransformer: BAAI/bge-base-en


2025-01-07 @ 17:43:33 | INFO     | src.embedding.embedding_models:get_bge_embedding:41 - Successfully loaded BGE embedding model


INFO | 2025-01-07T17:43:33Z | Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


2025-01-07 @ 17:43:34 | WARNING  | src.embedding.vector_store:retrieve_docs:95 - Retrieved 5 documents
Seeking guidance for helping my parents Helpo. So my dad is my mother's (has Alzheimers) primary care giver and has not been taking the best care of her.  I know he's exhausted and depressed and is in very bad health.  He recently had a fall and broke his ankle and is in rehab.  One of my younger brothers is looking after my mom briefly when he's not at work but shes in the house by herself for most of the day (even before the injury).  My mother needs to be in an assisted living facility but I don't think they have the money given how much they get from social security and medicare.  They had equity in their house but I think my father has already pulled it out and spent it.  He's very shady about his finances and it's almost impossible to get him to be honest about his entire situation so we can figure out a short term and long term plan for them.  With this recent injury I drove up

### Retrieve grading 
*Preventing hallucination and erroneous retrieval that is not relevant to the question but based on the idiosyncrasies of the embedding model or chunking.*

---------------------

In [ ]:
from typing import List
from retrieval_grading import grade_retrieval

# Grade the retrieval results
graded_docs = grade_retrieval(query_text, similarity_search_res)

# Print graded results
for doc in graded_docs:
    print(f"Document content: {doc}")


### Generate answer
*Using the graded documents to generate an answer.*

--------------------------------

In [ ]:
from answer_generation import generate_answer

answer = generate_answer(query_text, similarity_search_res)

print(answer)